In [3]:
import os
import random
import imutils as im
import numpy as np
from cv2 import cv2

img_dir = 'data/plates/test/synth_origin/'
out_dir = 'data/plates/test/synth_warped/'
img_list = os.listdir(img_dir)

In [4]:
from tqdm.auto import tqdm

for i in tqdm(img_list):
    img = cv2.imread(img_dir + i)
    rows, cols = img.shape[:2]
    x = random.randint(-rows//4, rows//4)
    space = abs(x) + 10
    x1, y1, x2, y2 = 0, 0, cols, rows
    pts1 = np.float32([[x1,y1], [x1,y2], [x2, y1], [x2,y2]]) # lt, lb, rt, rb
    pts2 = np.float32([[x1 + space, y1 + x + space], [x1 + space, y2 + x - space], [x2 - space, y1 - x + space], [x2 - space,y2 -x - space]])#
    mtrx = cv2.getPerspectiveTransform(pts1, pts2)
    dst = cv2.warpPerspective(img, mtrx, (cols, rows))
    cv2.imwrite(out_dir + i, dst)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [4]:
origin_img_path = 'data/plates/train/a/'
origin_imgs = os.listdir(origin_img_path)
image = cv2.imread(origin_img_path + origin_imgs[46513])

In [5]:
# Resize the image - change width to 500
newwidth = 500
image = im.resize(image, width=newwidth)

# RGB to Gray scale conversion
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Noise removal with iterative bilateral filter(removes noise while preserving edges)
d, sigmaColor, sigmaSpace = 11,17,17
filtered_img = cv2.bilateralFilter(gray, d, sigmaColor, sigmaSpace)

# Find Edges of the grayscale image
lower, upper = 170, 200
edged = cv2.Canny(filtered_img, lower, upper)

# Find contours based on Edges
cnts,hir = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

cnts=sorted(cnts, key = cv2.contourArea, reverse = True)[:10]
NumberPlateCnt = None
print("Number of Contours found : " + str(len(cnts)))


# loop over our contours to find the best possible approximate contour of number plate
count = 0
for c in cnts:
        peri = cv2.arcLength(c, True)

        epsilon = 0.01 * peri
        approx = cv2.approxPolyDP(c, epsilon, True)

        if len(approx) == 4:  # Select the contour with 4 corners
            print(approx)
            NumberPlateCnt = approx #This is our approx Number Plate Contour
            break

# Display the original image
cv2.imshow("Input Image", image)
# Display Grayscale image
cv2.imshow("Gray scale Image", gray)
# Display Filtered image
cv2.imshow("After Applying Bilateral Filter", filtered_img)
# Display Canny Image
cv2.imshow("After Canny Edges", edged)
# Drawing the selected contour on the original image
cv2.drawContours(image, [NumberPlateCnt], -1, (255,0,0), 2)
cv2.imshow("Output", image)

cv2.waitKey(1) #Wait for user input before closing the images displayed
cv2.destroyAllWindows()

Number of Contours found : 10


error: OpenCV(4.4.0) /tmp/pip-req-build-re6mpwzb/opencv/modules/imgproc/src/drawing.cpp:2598: error: (-215:Assertion failed) reader.ptr != NULL in function 'cvDrawContours'
